# Common methods

In [4]:
import os

# Function to normalize strings: remove spaces, convert to uppercase, and remove hyphens
def normalize_string(s):
    return s.replace(" ", "").replace("-", "").upper().strip().replace("/","").replace("\n","").replace("_","")

def readFile(filemane):
    data = []
    with open(filemane, 'r') as file:
        csvFile = csv.reader(file)
        for lines in csvFile:
            data.append(lines)
    data.pop(0)
    #print(data[0])
    return data

def checkRepeated(a):
    # A set to keep track of elements that have been seen
    seen = set()
    # A list to store duplicates found in the input list
    duplicates = []

    # Iterate over each element in the list
    for i in a:
        if i in seen:
            duplicates.append(i)
        else:
            seen.add(i)

    if len(duplicates) > 0:
        print(f"List has duplicates {duplicates}")
    #return duplicates
    
    
def cleanFolderOrCreateIt(folderPath):
    # Check if the directory exists
    if not os.path.exists(folderPath):
        # Create the directory if it doesn't exist
        os.makedirs(folderPath)
    else:
        for filename in os.listdir(folderPath):
            file_path = os.path.join(folderPath, filename)
            if os.path.isfile(file_path):
                try:
                    os.remove(file_path)
                except Exception as e:
                    print(f'Error eliminando {file_path}: {e}')
                    
                    
# Function to sanitize filenames
def sanitize_filename(name):
    name = name.lower()
    # Special case: if name matches certain criteria, assign fixed filename
    if "cisco" in name:
        filename = "cisco.json"
    else:
        # Remove or replace problematic characters
        name = re.sub(r'[^\w\s-]', '', name)  # Remove special characters except underscores and hyphens
        name = re.sub(r'\s+', '_', name)      # Replace spaces with underscores
        filename = f"{name}.json"
    return filename

def updateFile(dst_path_file,proccessed_json):
        # Check if the file exists
    if os.path.exists(dst_path_file):
        # If it exists, you might want to load and update it, or just overwrite
        with open(dst_path_file, 'r', encoding='utf-8') as f:
            existing_data = json.load(f)
            existing_data.update(proccessed_json)
    else:
        existing_data = proccessed_json

    # Write the data to the file
    with open(dst_path_file, 'w', encoding='utf-8') as f:
        json.dump(existing_data, f, ensure_ascii=False, indent=4)

# EXTRACTION (skippable): Extract reports dates list from ORKL

In [16]:
# 1a933813-3deb-4d6f-8e0f-33b9187970f9.json-["reports"][X]["report_names"][0] - "2017-01-22 - OurMine.pdf"
import re, os, json, csv
import pandas as pd
import networkx as nx
import os
import json
from networkx.algorithms.connectivity import local_edge_connectivity
import shutil
import ast
import copy


path_raw_data = "0-raw-data/"
dst_path = "3-nameclusters/"
path = path_raw_data+"ORKL/TAs/jsons/" 
jsons = []

report_list = []
file_creation_list = []

for file in os.listdir(path):
    if file.endswith(".json"):
        file_path = os.path.join(path, file)
        with open(file_path, "r", encoding="utf-8") as f:
            try:
                content = json.load(f)
                
                ta_aliases = []
                if content["aliases"] != None:
                    ta_aliases = content["aliases"]
                #print(ta_aliases)
                #print(content["id"])
                ta_aliases.append(content["main_name"])
                
                for report in content["reports"]:
                    report_creation_date = report["file_creation_date"]
                    if report_creation_date == "0001-01-01T00:00:00Z":
                        report_creation_date = report["created_at"]
                    # [TA-id, report name, report creation date, report_id, TA_aliases]
                    report_list.append([content["id"],report["report_names"][0],report_creation_date,report["id"],content["aliases"]])
                    #report_list.append(report["report_names"][0])
                    #file_creation_list.append(report["file_creation_date"])
            except json.JSONDecodeError as e:
                print(f"Error en {file}: {e}")

print("Done3")
# --> En este punto tenemos los reportes (report_list), con el TA ID y los aliases que hay que comprobar, incluyendo en main_name
# print(report_list)

dates_list = []

# Expresiones regulares para distintos formatos de fecha
regex_date_1 = re.compile(r"\d{4}-\d{2}-\d{2}")     # YYYY-MM-DD
regex_date_2 = re.compile(r"\d{2}-\d{2}-\d{4}")     # MM-DD-YYYY
regex_anio     = re.compile(r"\b(19|20)\d{2}\b")     # Año suelto (1900–2099)

# apuntamos aquellos aliases para los que hemos encontrado report que los menciona
#not_mentioned_alias = []
all_aliases = []
mentioned_alias = []
for file in range(0,len(report_list)):
    date = None
    ta_id = report_list[file][0]
    
    # Buscar formatos completos
    match1 = regex_date_1.search(report_list[file][1])
    match2 = regex_date_2.search(report_list[file][1])
    match3 = regex_anio.search(report_list[file][1])
    
    if match1 and match1.group().split(".")[0] != "1901": # Buscar YYYY-MM-DD
        date = match1.group()
        
    elif match2 and match2.group().split(".")[0] != "1901": # Buscar MM-DD-YYYY
        date = match2.group()
        month,day,year = date.split("-")
        date = year+"-"+month+"-"+day
    elif match3 and match3.group().split(".")[0] != "1901": # Si no hay fecha completa, buscar un año
        date = match3.group()
        
    else:
        #"file_creation_date": "2021-10-07T01:42:13Z",
        # Si no hay fecha completa, buscar un año en
        #date = file_creation_list[file].split("T")[0]
        date = report_list[file][2].split("T")[0]
        #print(date)
    
    # --> Aqui tenemos la fecha mas apropiada del reporte (date)
    
    
    # Buscamos algun alias mencionado en el reporte con report_id
    report_id = report_list[file][3]
    ta_aliases = report_list[file][4]
    if ta_aliases == None:
        ta_aliases = []
    report_data = ""
    
    # Leemos los datos en texto plano del reporte
    with open(path_raw_data+"ORKL/library/jsons_entries/"+report_id+".json", "r", encoding="utf-8") as f:
        try:
            content = json.load(f)
            #report_data = content["plain_text"].upper().replace("-", "")
            report_data = (content["plain_text"] + content["title"] + content["report_names"][0] + content["references"][0]).upper().replace("-", " ")
            
            report_vendor = content["authors"]
            if report_vendor == "":
                report_vendor = "Unknown source"
        except json.JSONDecodeError as e:
            print(f"Error en {file}: {e}")
    
    # Buscamos el alias en el reporte
    for alias in ta_aliases:
        
        #not_mentioned_alias.append(ta_id+","+alias)
        cmp_alias = alias
        
        # Separamos el alias en palabras
        words = alias.split()
        # Si tiene mas de dos palabras, cogemos las primeras dos
        if len(words) >= 2:
            cmp_alias = " ".join(words[:2])
        
        
        if cmp_alias.upper().replace("-", " ") in report_data:
            dates_list.append([ta_id,date,alias,report_id,report_vendor])
            mentioned_alias.append(ta_id+","+alias)

print("Done2")
# Comprobamos aquellos aliases que no se mencionan en los reportes y les ponemos la ultima fecha

for file in range(0,len(report_list)):
    date = None
    ta_id = report_list[file][0]
    report_id = report_list[file][3]
    ta_aliases = report_list[file][4]
    if ta_aliases == None:
        ta_aliases = []

    for alias in ta_aliases:
        if ta_id+alias not in mentioned_alias:
            dates_list.append([ta_id,None,alias,None,None])#[ta_id,date,alias,report_id,report_vendor]
            mentioned_alias.append(ta_id+alias)


'''#print(dates_list)
with open("dates_list.txt", "w") as f:
    f.write(str(dates_list))
    
# Save data to file
filename = "dates_list.csv"
with open(filename,"w") as f:
    fieldnames = [["ta_id","date","alias","report_id","report_vendor"]]
    
    writer = csv.writer(f)
    writer.writerows(fieldnames)
    writer.writerows(dates_list)
    print(filename+" created")'''
    
print("Done1")



###############################################################################################

CRWD_report_date = "2025-06-02"
CRWD_report_id = "https://www.crowdstrike.com/en-us/blog/crowdstrike-and-microsoft-unite-to-deconflict-cyber-threat-attribution/"
CRWD_report_vendor = "Crowdstrike"
df = pd.read_excel(path_raw_data+'/CrowdStrike-adversary-deconfliction.xlsx')

# Convert DataFrame to NumPy array using .values
deconfliction_data = df.values



#cloned = copy.deepcopy(original)
temp_dates_list = copy.deepcopy(dates_list)

for data in dates_list:
    # [["ta_id","date","alias","report_id","report_vendor"]]
    ta_alias = data[2]
    ta_id = data[0]
    
    for dec_data in deconfliction_data:
        
        if ta_alias.upper() == dec_data[0].upper() or ta_alias.upper() == dec_data[1].upper():
            if [ta_id,CRWD_report_date,dec_data[1].upper(),CRWD_report_id,CRWD_report_vendor] not in temp_dates_list:
                temp_dates_list.append([ta_id,CRWD_report_date,dec_data[1].upper(),CRWD_report_id,CRWD_report_vendor])
                #print([ta_id,CRWD_report_date,dec_data[1].upper(),CRWD_report_id,CRWD_report_vendor])
                #print(f"CRWD {ta_id}: {ta_alias.upper()}, {dec_data[1].upper()}")
                #print(data)
                #print()
            if [ta_id,CRWD_report_date,dec_data[0].upper(),CRWD_report_id,CRWD_report_vendor] not in temp_dates_list:
                temp_dates_list.append([ta_id,CRWD_report_date,dec_data[0].upper(),CRWD_report_id,CRWD_report_vendor])
                #print([ta_id,CRWD_report_date,dec_data[0].upper(),CRWD_report_id,CRWD_report_vendor])
                #print(f"CRWD {ta_id}: {ta_alias.upper()}, {dec_data[0].upper()}")
                #print(data)
                #print()


###############################################################################################

MSFT_report_date = "2025-06-02"
MSFT_report_id = "https://learn.microsoft.com/en-us/unified-secops-platform/microsoft-threat-actor-naming"
MSFT_report_vendor = "Microsoft"

dates_list2 = copy.deepcopy(temp_dates_list)
#temp_dates_list = dates_list.copy()  

# Open and read the JSON file
with open(path_raw_data+"/MicrosoftMapping.json", 'r') as file:
    msft_data = json.load(file)

for data in temp_dates_list:
    # [["ta_id","date","alias","report_id","report_vendor"]]
    ta_alias = data[2]
    ta_id = data[0]
    
    for ta in msft_data:
        ta_name = ta["Threat actor name"]
        ta_other_names = ta["Other names"].split(", ")
        #ta_other_names = ta["Other names"].
        if  ta_other_names == [""]:
            ta_other_names = []

        if ta_alias == ta_name or ta_alias in ta_other_names:
            
            if [ta_id,MSFT_report_date,ta_name,MSFT_report_id,MSFT_report_vendor] not in dates_list2:
                dates_list2.append([ta_id,MSFT_report_date,ta_name,MSFT_report_id,MSFT_report_vendor])
                #print([ta_id,MSFT_report_date,ta_name,MSFT_report_id,MSFT_report_vendor])
            for name in ta_other_names:
                if [ta_id,MSFT_report_date,name,MSFT_report_id,MSFT_report_vendor] not in dates_list2:
                    dates_list2.append([ta_id,MSFT_report_date,name,MSFT_report_id,MSFT_report_vendor])
                    #print([ta_id,MSFT_report_date,name,MSFT_report_id,MSFT_report_vendor])
            break
        
#print(dates_list)
#with open("dates_list2.txt", "w") as f:
#    f.write(str(dates_list2))
    
print("DONE")

saved_dates_list = copy.deepcopy(dates_list)
dates_list = copy.deepcopy(dates_list2)

# Save data to file
filename = "0-raw-data/dates_list2.csv"
with open(filename,"w") as f:
    fieldnames = [["ta_id","date","alias","report_id","report_vendor"]]
    
    writer = csv.writer(f)
    writer.writerows(fieldnames)
    writer.writerows(dates_list2)
    print(filename+" created")

Done3
Done2
Done1
DONE
0-raw-data/dates_list2.csv created


# EXTRACTION (skippable): Malpedia malware list scrapping

In [27]:
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

# Base URL for actor pages
base_url = "https://malpedia.caad.fkie.fraunhofer.de"

# URL of the webpage to fetch
webpage_url = "https://malpedia.caad.fkie.fraunhofer.de/actors"

actor_aliases_dict = {}

filename = "malpedia_mapping.json"
path_raw_data = "0-raw-data/"
src_path_file = path_raw_data + filename

try:
    # Fetch the main webpage
    response = requests.get(webpage_url)
    response.raise_for_status()
    webpage_content = response.text

    # Parse with BeautifulSoup
    soup = BeautifulSoup(webpage_content, 'html.parser')

    # Pattern for data-href="/actor/XXXX"
    pattern = re.compile(r'^/actor/\w+$')

    # Find all actor links
    actor_links = []
    for element in soup.find_all(attrs={"data-href": True}):
        data_href = element['data-href']
        if pattern.match(data_href):
            actor_links.append(data_href)

            
    #for link in actor_links:
    #    print(link)
    # Use tqdm for progress bar over actor links
    for link in tqdm(actor_links, desc="Processing actors"):
        actor_url = base_url + link
        try:
            actor_response = requests.get(actor_url)
            actor_response.raise_for_status()
            actor_soup = BeautifulSoup(actor_response.text, 'html.parser')

            # Extract actor's name from URL (the part after the last "/")
            actor_name = link.split('/')[-1]

            aliases = []

            # Find all divs; look for one starting with "aka:"
            for div in actor_soup.find_all('div'):
                text = div.get_text(strip=True)
                if text.lower().startswith("aka:"):
                    # Remove "aka:" prefix and split aliases by comma or semicolon
                    alias_text = text[4:].strip()  # Remove "aka:"
                    # Split by comma or semicolon if multiple aliases
                    aliases = re.split(r'[;,]', alias_text)
                    # Clean up whitespace
                    aliases = [alias.strip() for alias in aliases if alias.strip()]
                    break

            # Add to dictionary
            actor_aliases_dict[actor_name] = aliases
            #print(f"{actor_name}:{aliases}")

        except requests.RequestException as e:
            print(f"Error fetching {actor_url}: {e}")

    # Optional: print the dictionary
    # for actor, aliases in actor_aliases_dict.items():
    #     print(f"{actor}: {aliases}")

except requests.RequestException as e:
    print(f"Error fetching the main webpage: {e}")


with open(src_path_file, 'w', encoding='utf-8') as f:
    json.dump(actor_aliases_dict, f, ensure_ascii=False, indent=4)
# At the end, you can print or save the dictionary
#print("\nCollected actor aliases:")
#for actor, aliases in actor_aliases_dict.items():
#    print(f"{actor}: {aliases}")

Processing actors:   6%|█▍                     | 47/755 [01:48<27:16,  2.31s/it]


KeyboardInterrupt: 

# EXTRACTION (skippable): Extract palo alto dictionary

In [ ]:
import requests
from bs4 import BeautifulSoup
import re,json
from tqdm import tqdm

# Base URL for actor pages
base_url = "https://malpedia.caad.fkie.fraunhofer.de"

# URL of the webpage to fetch
webpage_url = "https://malpedia.caad.fkie.fraunhofer.de/actors"

actor_malware_dict = {}

filename = "malpedia_ta_malware.json"
src_path_file = path_raw_data + filename

try:
    # Fetch the main webpage
    response = requests.get(webpage_url)
    response.raise_for_status()
    webpage_content = response.text

    # Parse with BeautifulSoup
    soup = BeautifulSoup(webpage_content, 'html.parser')

    # Pattern for data-href="/actor/XXXX"
    pattern = re.compile(r'^/actor/\w+$')

    # Find all actor links
    actor_links = []
    for element in soup.find_all(attrs={"data-href": True}):
        data_href = element['data-href']
        if pattern.match(data_href):
            actor_links.append(data_href)

            
    #for link in actor_links:
    #    print(link)
    # Use tqdm for progress bar over actor links
    for link in tqdm(actor_links, desc="Processing actors"):
        actor_url = base_url + link
        try:
            actor_response = requests.get(actor_url)
            actor_response.raise_for_status()
            actor_soup = BeautifulSoup(actor_response.text, 'html.parser')

            # Extract actor's name from URL (the part after the last "/")
            actor_name = link.split('/')[-1]
            actor_name = normalize_string(actor_name)

            malware_list = []
            
            for div in actor_soup.find_all("span", class_="badge-custom clickable-row"):
                text = div.get_text(strip=True)
                #print(text)
                malware_list.append(text)
                  
        
            # Add to dictionary
            actor_malware_dict[actor_name] = malware_list
            #print(f"{actor_name}:{malware_list}")

        except requests.RequestException as e:
            print(f"Error fetching {actor_url}: {e}")

    # Optional: print the dictionary
    # for actor, aliases in actor_aliases_dict.items():
    #     print(f"{actor}: {aliases}")

except requests.RequestException as e:
    print(f"Error fetching the main webpage: {e}")


with open(src_path_file, 'w', encoding='utf-8') as f:
    json.dump(actor_malware_dict, f, ensure_ascii=False, indent=4)
# At the end, you can print or save the dictionary
#print("\nCollected actor aliases:")
#for actor, aliases in actor_aliases_dict.items():
#    print(f"{actor}: {aliases}")

# NORMALIZATION: Generate taxonomies

In [42]:
import requests, json
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import pandas as pd

path_raw_data = "0-raw-data/"
path_proccesed_data = "1-taxonomies/"
sufix = "_taxonomy"


unk_json = {}

cleanFolderOrCreateIt(path_proccesed_data)
#####################################################################################
# ORKL
#####################################################################################

filename = "orkl_tas.json"
dst_filename = "orkl"+sufix+".json"
src_path_file = path_raw_data + filename
dst_path_file = path_proccesed_data + dst_filename

with open(src_path_file) as f:
    data = json.load(f)
    
sources = []
aliases = []
for ta in data:    
    sources.append(ta["source_name"].split(":")[0])
    ta_aliases = ta["aliases"]
    if ta_aliases != None and len(ta_aliases) > 0 and ta_aliases != [""]:
        for alias in ta_aliases:
            #aliases.append(alias)
            unk_json[alias] = normalize_string(alias)
    
for s in sources:
    proccessed_json = {}
    for ta in data:
        source_name = ta["source_name"].split(":")[1]
        ta_source = ta["source_name"].split(":")[0]
        if s == ta_source:
            #normalized_name = source_name.upper().replace(" ","").replace("-","")
            normalized_name = normalize_string(source_name)

            proccessed_json[source_name] = normalized_name
        
                
        
        
    dst_path_file = path_proccesed_data + s.lower() + sufix + ".json"
    with open(dst_path_file, 'w', encoding='utf-8') as f:
        json.dump(proccessed_json, f, ensure_ascii=False, indent=4)

'''proccessed_json = {}
for ta in data:
    source_name = ta["source_name"]
    normalized_name = source_name.upper().replace(" ","").replace("-","")
    
    proccessed_json[source_name] = normalized_name

with open(dst_path_file, 'w', encoding='utf-8') as f:
    json.dump(proccessed_json, f, ensure_ascii=False, indent=4) '''
    
#####################################################################################
# MICROSOFT
#####################################################################################
    
filename = "MicrosoftMapping.json"
dst_filename = "microsoft"+".json"
src_path_file = path_raw_data + filename
dst_path_file = path_proccesed_data + dst_filename

with open(src_path_file) as f:
    data = json.load(f)
    
proccessed_json = {}
for ta in data:
    source_name = ta["Threat actor name"]
    #normalized_name = source_name.upper().replace(" ","").replace("-","")
    normalized_name = normalize_string(source_name)
    proccessed_json[source_name] = normalized_name
    
    aliases = ta["Other names"].split(", ")
    for alias in aliases:
        unk_json[alias] = normalize_string(alias)
with open(dst_path_file, 'w', encoding='utf-8') as f:
    json.dump(proccessed_json, f, ensure_ascii=False, indent=4)
    
    
#####################################################################################
# MISP (ya incluido en ORKL)
#####################################################################################

filename = "misp-threat-actor.json"
dst_filename = "misp"+sufix+".json"
src_path_file = path_raw_data + filename
dst_path_file = path_proccesed_data + dst_filename

with open(src_path_file) as f:
    data = json.load(f)
    data = data["values"]
    
proccessed_json = {}
for ta in data:
    source_name = ta["value"]
    #normalized_name = source_name.upper().replace(" ","").replace("-","")
    normalized_name = normalize_string(source_name)
    
    proccessed_json[source_name] = normalized_name
    
    
    if "meta" in ta:
        if "synonyms" in ta["meta"]:
            aliases = ta["meta"]["synonyms"]
            for alias in aliases:
                unk_json[alias] = normalize_string(alias)
src_path_file = path_proccesed_data + "mispgalaxy_taxonomy.json"
with open(src_path_file) as f:
    data = json.load(f)
    proccessed_json.update(data)
    
with open(src_path_file, 'w', encoding='utf-8') as f:
    json.dump(proccessed_json, f, ensure_ascii=False, indent=4)
    
    
    
    


##################################################################################


    
#unk_json = {}
path_raw_data = "0-raw-data/"
path_proccesed_data = "1-taxonomies/"

filename = "malpedia_tas.json"
src_path_file = path_raw_data + filename
    
with open(src_path_file) as f:
    data = json.load(f)
    
for d in data:
    unk_json[d] = normalize_string(d)
    
    for alias in data[d]:
        unk_json[alias] = normalize_string(alias)

        
        
#################################################################

# dict_tas_paloalto.json

path_raw_data = "0-raw-data/"
path_proccesed_data = "1-taxonomies/"
sufix = "_taxonomy"

filename = "dict_tas_paloalto.json"
src_path_file = path_raw_data + filename
dst_path_file = path_proccesed_data + "paloaltounit42.json"


proccessed_json = {}


with open(src_path_file) as f:
    data = json.load(f)

for ta in data:
    #print(ta)
    aliases = data[ta]["Also Known As"]
    #normalized_name = source_name.upper().replace(" ","").replace("-","")
    normalized_name = normalize_string(ta)
    
    proccessed_json[ta] = normalized_name    


    for alias in aliases:
        unk_json[alias] = normalize_string(alias)


updateFile(dst_path_file,proccessed_json)
    
#################################################################


    
with open(path_proccesed_data+"unk.json", 'w', encoding='utf-8') as f:
    json.dump(unk_json, f, ensure_ascii=False, indent=4)
    
    
print("DONE")







DONE


In [43]:
import requests, json,os
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import pandas as pd

#####################################################################################
# APT Groups and Operations Excel (https://docs.google.com/spreadsheets/d/1H9_xaxQHpWaa4O_Son4Gx0YOIzlcBWMsdvePFX68EKU/edit?gid=361554658#gid=361554658)
#####################################################################################

path_raw_data = "0-raw-data/"
path_proccesed_data = "1-taxonomies/"
sufix = "_taxonomy"

filename = "APT Groups and Operations.xlsx"
src_path_file = path_raw_data + filename
dst_path_file = path_proccesed_data + dst_filename


        
# Load a specific sheet by name
#df_china = pd.read_excel('src_path_file', sheet_name='China')

for x in range(1,10):
    # Or load a sheet by index (0-based)
    df = pd.read_excel(src_path_file, sheet_name=x, header=1)

    '''# For each column, list the non-NaN elements
    non_nan_elements = {col: df[col].dropna().tolist() for col in df.columns}

    # Print the results
    for col, elements in non_nan_elements.items():
        print(f"Column: {col}")
        print(elements)
        print()'''
    
    #print(df.columns.tolist())
    
    # Find the index of "Common Name"
    start_idx = df.columns.get_loc("Common Name")

    # Find the first column containing "operation" (case-insensitive)
    operation_idx = None
    for i, col in enumerate(df.columns):
        if "operation" in col.lower():
            operation_idx = i
            break

    # Determine the range of columns to process
    if operation_idx is None:
        columns_to_process = df.columns[start_idx + 1:]
    else:
        columns_to_process = df.columns[start_idx + 1:operation_idx]

    

    # Iterate through each column
    for col in columns_to_process:
        filename = sanitize_filename(col).lower().replace("_","")
        filepath = os.path.join(path_proccesed_data, filename)

        # Load existing data if file exists
        if os.path.exists(filepath):
            with open(filepath, "r") as f:
                try:
                    json_data = json.load(f)
                except json.JSONDecodeError:
                    json_data = {}
        else:
            json_data = {}

        # Update the JSON data with current column values
        for idx, value in df[col].items():
            if pd.isna(value):
                continue
                
            if "," in str(value):
                namesList = str(value).split(",")
                
                for n in namesList:
                    if n.replace(" ","") not in json_data.keys():
                        #normalized_value = str(n).upper().replace(" ", "").replace("-", "")
                        normalized_value = normalize_string(str(n))
                        json_data[str(n).strip()] = normalized_value
            else:
                if str(value).replace(" ","") not in json_data.keys():
                    #normalized_value = str(value).upper().replace(" ", "").replace("-", "")
                    normalized_value = normalize_string(str(value))
                    json_data[str(value).strip()] = normalized_value
        
        
        updateFile(filepath,json_data)
        # Save the updated JSON data back to file
        #with open(filepath, "w") as f:
        #    json.dump(json_data, f, indent=4)

        print(f"Processed {filepath}")
    



#################################################################


print("DONE")

Processed 1-taxonomies/crowdstrike.json
Processed 1-taxonomies/irl.json
Processed 1-taxonomies/kaspersky.json
Processed 1-taxonomies/secureworks.json
Processed 1-taxonomies/mandiant.json
Processed 1-taxonomies/fireeye.json
Processed 1-taxonomies/symantec.json
Processed 1-taxonomies/isight.json
Processed 1-taxonomies/cisco.json
Processed 1-taxonomies/paloaltounit42.json
Processed 1-taxonomies/othernames.json
Processed 1-taxonomies/mitreattck.json
Processed 1-taxonomies/othername1.json
Processed 1-taxonomies/othername2.json
Processed 1-taxonomies/othername3.json
Processed 1-taxonomies/othername4.json
Processed 1-taxonomies/othername5.json
Processed 1-taxonomies/othername6.json
Processed 1-taxonomies/othername7.json
Processed 1-taxonomies/othername8.json
Processed 1-taxonomies/othername9.json
Processed 1-taxonomies/othername10.json
Processed 1-taxonomies/othername11.json
Processed 1-taxonomies/othername12.json
Processed 1-taxonomies/mitreattck.json
Processed 1-taxonomies/secureworks.json


# NORMALIZATION: Clean files

In [44]:
import json
import os
import re
import glob

# Define folder path
folder_path = '1-taxonomies'



def updateFile(dst_path_file,proccessed_json):
        # Check if the file exists
    if os.path.exists(dst_path_file):
        # If it exists, you might want to load and update it, or just overwrite
        with open(dst_path_file, 'r', encoding='utf-8') as f:
            existing_data = json.load(f)
            existing_data.update(proccessed_json)
    else:
        existing_data = proccessed_json

    # Write the data to the file
    with open(dst_path_file, 'w', encoding='utf-8') as f:
        json.dump(existing_data, f, ensure_ascii=False, indent=4)
        
def load_or_create_json(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            try:
                data = json.load(f)
            except json.JSONDecodeError:
                # If the file exists but is empty or invalid, return empty dict
                data = {}
    else:
        data = {}
        with open(file_path, 'w') as f:
            json.dump(data, f)
    return data

# Function to process entries that are strings with multiple codes
def process_string_entry(entry):
    if isinstance(entry, str):
        clean_entry = entry.strip().strip('"')
        # Check for newline separated codes
        if '\n' in clean_entry:
            parts = clean_entry.split('\n')
            codes = []
            for part in parts:
                code_match = re.match(r'(G\d+)', part, re.IGNORECASE)
                if code_match:
                    code = code_match.group(1).upper()
                    codes.append((code, code))
            return dict(codes)
        # Check for comma-separated codes
        elif ',' in clean_entry:
            parts = [p.strip() for p in clean_entry.split(',')]
            codes = []
            for part in parts:
                code_match = re.match(r'(G\d+)', part, re.IGNORECASE)
                if code_match:
                    code = code_match.group(1).upper()
                    codes.append((code, code))
            return dict(codes)
    return None

# Transform data: process string entries into dicts with individual codes
def transform_data(data):
    if isinstance(data, dict):
        new_dict = {}
        for key, value in data.items():
            processed = process_string_entry(value)
            if processed:
                new_dict.update(processed)
            else:
                new_dict[key] = value
        return new_dict
    elif isinstance(data, list):
        new_list = []
        for item in data:
            if isinstance(item, dict):
                new_list.append(transform_data(item))
            elif isinstance(item, str):
                processed = process_string_entry(item)
                if processed:
                    new_list.append(processed)
                else:
                    new_list.append(item)
        return new_list
    else:
        return data

    
    
# File paths
file1_path = os.path.join(folder_path, 'mitreatck.json')
file2_path = os.path.join(folder_path, 'mitreattck.json')
output_path = os.path.join(folder_path, 'mitre.json')

# Load JSON files
with open(file1_path, 'r', encoding='utf-8') as f:
    data1 = json.load(f)
with open(file2_path, 'r', encoding='utf-8') as f:
    data2 = json.load(f)

# Merge data
if isinstance(data1, list) and isinstance(data2, list):
    combined_data = data1 + data2
elif isinstance(data1, dict) and isinstance(data2, dict):
    combined_data = {**data1, **data2}
else:
    combined_data = [data1, data2]


# Apply transformation
transformed_data = transform_data(combined_data)




temp_data0 = transformed_data.copy()
to_del = []

for d in transformed_data:
    if "\n" in d:
        data = re.findall(r'G\d{4}', temp_data0[d])
        #data = temp_data0[d].split('\n')
        
        pattern = r'^G\d+'
        print(data)
        for x in data:
            if re.match(pattern, x):
                temp_data0[x] = x
        to_del.append(d)
        
transformed_data = temp_data0.copy()
for d in to_del:
    del transformed_data[d]

# Save the combined and transformed data
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(transformed_data, f, indent=2)

print(f"Combined and transformed data saved to {output_path}")

# Delete source files
try:
    os.remove(file1_path)
    os.remove(file2_path)
    print("Source files deleted successfully.")
except Exception as e:
    print(f"Error deleting source files: {e}")
    
    
    
##################################################################

# Define folder path
folder_path = '1-taxonomies'

# Find all JSON files starting with 'othername'
othername_files = glob.glob(os.path.join(folder_path, 'othername*.json'))

combined_data = []

# Load and collect all 'othername' JSON files
for filepath in othername_files:
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
        combined_data.append(data)

# Optional: Depending on the structure, you might want to merge dicts or lists
# Here, assuming the files contain dictionaries, merge them
merged_result = {}
for data in combined_data:
    if isinstance(data, dict):
        merged_result.update(data)
    elif isinstance(data, list):
        # If they are lists, extend the list
        if 'combined_list' not in locals():
            combined_list = []
        combined_list.extend(data)
    else:
        # Handle other data types if needed
        pass

# Decide what to save:
# If you want to save merged dict
output_data = merged_result

# Or, if you want to save combined list, uncomment:
# output_data = combined_list

corrected_result = {}

for data in output_data:
    groupNames = data.split(",")
    
    for g in groupNames:
        
        name1 = g
        pattern = r'\\u[0-9a-fA-F]{4}'
        
        if not re.search(r'[^\x00-\x7F]',  name1):
            
            if "(" in g:
                name1 = g.split("(")[0]
            if "–" in g:
                name1 = g.split("(")[0]
            if "/" in g:
                name1 = g.split("/")[0]
                name2 = g.split("/")[1]
                #corrected_result[name2.strip()] = name2.strip().upper().replace(" ","").replace("-","").replace("/","").replace("\n","")
                corrected_result[name2.strip()] = normalize_string(name2)
            
            #corrected_result[name1.strip()] = name1.strip().upper().replace(" ","").replace("-","").replace("/","").replace("\n","")
            corrected_result[name1.strip()] = normalize_string(name1)


# Save the combined data
#output_path = os.path.join(folder_path, 'unk.json')
#with open(output_path, 'w', encoding='utf-8') as f:
#    json.dump(corrected_result, f, indent=2)
    
#data0 = load_or_create_json(folder_path+'unk.json')


updateFile(folder_path+'/crowdstrike.json',corrected_result)







print(f"Combined 'othername' files saved to {output_path}")

# Delete the individual 'othername' files
for filepath in othername_files:
    try:
        os.remove(filepath)
        print(f"Deleted {filepath}")
    except Exception as e:
        print(f"Error deleting {filepath}: {e}")

#################################################################################

# Define folder path
folder_path = '1-taxonomies/'

# Load the first JSON file
with open(folder_path+'fireeye.json', 'r', encoding='utf-8') as file1:
    data1 = json.load(file1)

# Load the second JSON file
with open(folder_path+'fireeyename.json', 'r', encoding='utf-8') as file2:
    data2 = json.load(file2)

# Load the second JSON file
with open(folder_path+'isight.json', 'r', encoding='utf-8') as file3:
    data3 = json.load(file3)
    
# Merge the two dictionaries (or lists)
# If they are dictionaries:
merged_data = {**data1, **data2, **data3}
# If they are lists, use:
# merged_data = data1 + data2

for data in merged_data:
    if "\n" in merged_data[data]:
        merged_data[data] = merged_data[data].replace("\n","")


# Save the combined data to a new JSON file
with open(folder_path+'fireeye.json', 'w', encoding='utf-8') as outfile:
    json.dump(merged_data, outfile, ensure_ascii=False, indent=4)

os.remove(folder_path+'fireeyename.json')
os.remove(folder_path+'isight.json')

##################################################################################

# COMBINE secureworks y secureworks_taxonomy y dellsecurewokrs


with open(folder_path+'secureworks.json', 'r', encoding='utf-8') as file0:
    data0 = json.load(file0)
    
sourceFile = "secureworks_taxonomy.json"
# Load the first JSON file
with open(folder_path+sourceFile, 'r', encoding='utf-8') as file1:
    data1 = json.load(file1)
    data0.update(data1)
os.remove(folder_path+sourceFile)

sourceFile = "dellsecureworks.json"
# Load the first JSON file
with open(folder_path+sourceFile, 'r', encoding='utf-8') as file1:
    data1 = json.load(file1)
    data0.update(data1)
os.remove(folder_path+sourceFile)

with open(folder_path+'secureworks.json', 'w', encoding='utf-8') as file0:
    json.dump(data0, file0, ensure_ascii=False, indent=4)
    

##################################################################################
# COMBINE secureworks y secureworks_taxonomy y dellsecurewokrs


with open(folder_path+'talosgroup.json', 'r', encoding='utf-8') as file0:
    data0 = json.load(file0)
    
sourceFile = "cisco.json"
# Load the first JSON file
with open(folder_path+sourceFile, 'r', encoding='utf-8') as file1:
    data1 = json.load(file1)
    data0.update(data1)
os.remove(folder_path+sourceFile)


with open(folder_path+'talosgroup.json', 'w', encoding='utf-8') as file0:
    json.dump(data0, file0, ensure_ascii=False, indent=4)

##################################################################
# DUMP the SOURCES to the UNK json
# Load the first JSON file
with open(folder_path+'unk.json', 'r', encoding='utf-8') as file0:
    data0 = json.load(file0)

sourceFile = "mispgalaxy_taxonomy.json"
# Load the first JSON file
with open(folder_path+sourceFile, 'r', encoding='utf-8') as file1:
    data1 = json.load(file1)
    data0.update(data1)
os.remove(folder_path+sourceFile)

sourceFile = "mitre_taxonomy.json"
# Load the first JSON file
with open(folder_path+sourceFile, 'r', encoding='utf-8') as file1:
    data1 = json.load(file1)
    data0.update(data1)
os.remove(folder_path+sourceFile)

sourceFile = "repofkoreafsi.json"
# Load the first JSON file
with open(folder_path+sourceFile, 'r', encoding='utf-8') as file1:
    data1 = json.load(file1)
    data0.update(data1)
os.remove(folder_path+sourceFile)

sourceFile = "etda_taxonomy.json"
# Load the first JSON file
with open(folder_path+sourceFile, 'r', encoding='utf-8') as file1:
    data1 = json.load(file1)
    data0.update(data1)
os.remove(folder_path+sourceFile)

sourceFile = "crowdstrike.json"
# Load the first JSON file
with open(folder_path+sourceFile, 'r', encoding='utf-8') as file1:
    data1 = json.load(file1)
    data0.update(data1)
os.remove(folder_path+sourceFile)

sourceFile = "irl.json"
# Load the first JSON file
with open(folder_path+sourceFile, 'r', encoding='utf-8') as file1:
    data1 = json.load(file1)
    data0.update(data1)
os.remove(folder_path+sourceFile)

with open(folder_path+'unk.json', 'w', encoding='utf-8') as file0:
    json.dump(data0, file0, ensure_ascii=False, indent=4)


##################################################################################
print("DONE")





['G0103', 'G0107']
Combined and transformed data saved to 1-taxonomies/mitre.json
Source files deleted successfully.
Combined 'othername' files saved to 1-taxonomies/mitre.json
Deleted 1-taxonomies/othername12.json
Deleted 1-taxonomies/othername5.json
Deleted 1-taxonomies/othername2.json
Deleted 1-taxonomies/othername8.json
Deleted 1-taxonomies/othername10.json
Deleted 1-taxonomies/othername9.json
Deleted 1-taxonomies/othername4.json
Deleted 1-taxonomies/othername6.json
Deleted 1-taxonomies/othername11.json
Deleted 1-taxonomies/othername7.json
Deleted 1-taxonomies/othername3.json
Deleted 1-taxonomies/othernames.json
Deleted 1-taxonomies/othername1.json
DONE


In [45]:
#UPDATE from UNK JSON
folder_path = '1-taxonomies/'

elements = ["HYDROGEN", "HELIUM", "LITHIUM", "BERYLLIUM", "BORON", "CARBON", "NITROGEN", "OXYGEN", "FLUORINE", 
            "NEON", "SODIUM", "MAGNESIUM", "ALUMINUM", "SILICON", "PHOSPHORUS", "SULFUR", "CHLORINE", "ARGON", 
            "POTASSIUM", "CALCIUM", "SCANDIUM", "TITANIUM", "VANADIUM", "CHROMIUM", "MANGANESE", "IRON", "COBALT", 
            "NICKEL", "COPPER", "ZINC", "GALLIUM", "GERMANIUM", "ARSENIC", "SELENIUM", "BROMINE", "KRIPTON", 
            "RUBIDIUM", "STRONTIUM", "YTTRIUM", "ZIRCONIUM", "NIOBIUM", "MOLYBDENUM", "TECHNETIUM", "RUTHENIUM", 
            "RHODIUM", "PALLADIUM", "SILVER", "CADMIUM", "INDIUM", "ANTIMONY", "TELLURIUM", "IODINE", "XENON", 
            "CAESIUM", "BARIUM", "LANTHANUM", "CERIUM", "PRASEODYMIUM", "NEODYMIUM", "PROMETHIUM", "SAMARIUM", 
            "EUROPIUM", "GADOLINIUM", "TERBIUM", "DYSPROSIUM", "HOLMIUM", "ERBIUM", "THULIUM", "YTTERBIUM", 
            "LUTETIUM", "HAFNIUM", "TANTALUM", "TUNGSTEN", "RHENIUM", "OSMIUM", "IRIDIUM", "PLATINUM", "GOLD", 
            "MERCURY", "THALLIUM", "LEAD", "BISMUTH", "POLONIUM", "ASTATINE", "RADON", "FRANCIUM", "RADIUM", 
            "ACTINIUM", "THORIUM", "PROTACTINIUM", "URANIUM", "NEPTUNIUM", "PLUTONIUM", "AMERICIUM", "CURIUM", 
            "BERKELIUM", "CALIFORNIUM", "EINSTEINIUM", "FERMIUM", "MENDELEVIUM", "NOBELIUM", "LAWRENCIUM", 
            "RUTHERFORDIUM", "DUBNIUM", "SEABORGIUM", "BOHRIUM", "HASSIUM", "MEITNERIUM", "DARMSTADTIUM", 
            "ROENTGENIUM", "COPERNICIUM", "NIHONIUM", "FLEROVIUM", "MOSCOVIUM", "LIVERMORIUM", "TENNESSINE", 
            "OGANESSON"
]

animals = [ "PANDA","SPIDER","JACKAL","CHOLLIMA","BEAR","KITTEN","TIGER","SPHINX","LEOPARD"]


data0 = load_or_create_json(folder_path+'unk.json')
#temp_data0 = data0.copy()
temp_data0 = []

data_crwd = load_or_create_json(folder_path+'crowdstrike.json')
data_swrk = load_or_create_json(folder_path+'secureworks.json')
data_msft_new = load_or_create_json(folder_path+'microsoft.json')
data_mtr = load_or_create_json(folder_path+'mitre.json')
data_msft = load_or_create_json(folder_path+'microsoft_old.json')
data_360 = load_or_create_json(folder_path+'360.json')
data_mdt = load_or_create_json(folder_path+'mandiant.json')
data_tales = load_or_create_json(folder_path+'tales.json')
data_tencent = load_or_create_json(folder_path+'tencent.json')



data_recfut = load_or_create_json(folder_path+'recorded_future.json')
data_certua = {}


for d in data0:
    #print(data0[d])
    if data0[d] in elements:
        data_msft[d] = data0[d]
        #print(f"Found {data0[d]}")
        #del temp_data0[d]
        temp_data0.append(d)
        continue
    if "DEV-" in d.upper():
        data_msft[d] = data0[d]
        #print(f"Found {data0[d]}")
        #del temp_data0[d]
        temp_data0.append(d)
        continue
        
    for a in animals:
        if a in data0[d]:
            data_crwd[d] = data0[d]
            #print(f"Found {data0[d]}")
            #del temp_data0[d]
            temp_data0.append(d)
            continue
    
    #print(d)
    if "APTC" in data0[d]:
        data_360[d] = data0[d]
        #print(f"Found {data0[d]}")
        #del temp_data0[d]
        temp_data0.append(d)
        continue
    if "TG-" in d.upper():
        data_swrk[d] = data0[d]
        #print(f"Found {data0[d]}")
        #del temp_data0[d]
        temp_data0.append(d)
        continue
    pattern = r'^TG\d+'
    if re.match(pattern, data0[d]):
        data_swrk[d] = data0[d]
        #print(f"Found {data0[d]}")
        #del temp_data0[d]
        temp_data0.append(d)
        continue
        
    pattern = r'^APT\d+'
    if re.match(pattern, data0[d]):
        data_mdt[d] = data0[d]
        #print(f"Found {data0[d]}")
        #del temp_data0[d]
        temp_data0.append(d)
        continue
    pattern = r'^UNC\d+'
    if re.match(pattern, data0[d]):
        data_mdt[d] = data0[d]
        #print(f"Found {data0[d]}")
        #del temp_data0[d]
        temp_data0.append(d)
        continue
    pattern = r'^FIN\d+'
    if re.match(pattern, data0[d]):
        data_mdt[d] = data0[d]
        #print(f"Found {data0[d]}")
        #del temp_data0[d]
        temp_data0.append(d)
        continue
        
    if "UAC-" in d.upper() or "UAC_" in d.upper():
        data_certua[d] = data0[d]
        #del temp_data0[d]
        temp_data0.append(d)
        continue
    
    
    if "ATK" in d.upper():
        data_tales[d] = data0[d]
        #del temp_data0[d]
        temp_data0.append(d)
        continue
    if "TAG-" in d.upper() or "TAG_" in d.upper():
        data_recfut[d] = data0[d]
        #del temp_data0[d]
        temp_data0.append(d) 
        continue
        
    if "STORM" in data0[d] or "VOID" in data0[d]:
        data_msft_new[d] = data0[d]
        #print(f"Found {data0[d]}")
        #del temp_data0[d]
        temp_data0.append(d)
        continue
    
    pattern = r'^G\d+'
    if re.match(pattern, data0[d]):
        data_mtr[d] = data0[d]
        #print(f"Found {data0[d]}")
        #del temp_data0[d]
        temp_data0.append(d)
        continue
    if "TEMP." in data0[d]:
        data_mdt[d] = data0[d]
        #print(f"Found {data0[d]}")
        #print(f"Found {temp_data0[d]}")
        #del temp_data0[d]
        temp_data0.append(d)
        continue  
        
    if "T-APT-" in d.upper():
        data_tencent[d] = data0[d]
        #del temp_data0[d]
        temp_data0.append(d) 
        continue
    
        
#data0 = temp_data0.copy()
#print(temp_data0)
for d in temp_data0:
    if d in data0:
        del data0[d]
        
del data0[""]

with open(folder_path+'unk.json', 'w', encoding='utf-8') as file0:
    json.dump(data0, file0, ensure_ascii=False, indent=4)
updateFile(folder_path+'crowdstrike.json',data_crwd)
updateFile(folder_path+'secureworks.json',data_swrk)
updateFile(folder_path+'microsoft.json',data_msft_new)
updateFile(folder_path+'mitre.json',data_mtr)
updateFile(folder_path+'microsoft_old.json',data_msft)
updateFile(folder_path+'360.json',data_360)
updateFile(folder_path+'mandiant.json',data_mdt)
os.remove(folder_path+'tales.json')
updateFile(folder_path+'thales.json',data_tales)
updateFile(folder_path+'recorded_future.json',data_recfut)
updateFile(folder_path+'cert_ua.json',data_certua)
updateFile(folder_path+'tencent.json',data_tencent)
    
##################################################################################
print("Done")

Done


In [46]:
# Order taxonomies alphabetically
import os
import json

# Carpeta que contiene los archivos JSON
folder_path = '1-taxonomies'

# Lista todos los archivos en la carpeta
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        
        # Leer el contenido JSON
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
         
        unique_values = {}
        seen_values = set()
        for key, value in data.items():
            if value not in seen_values:
                unique_values[key] = value
                seen_values.add(value)
                
        data = unique_values
        
        # Ordenar alfabéticamente por claves
        sorted_data = dict(sorted(data.items()))
        
        # Guardar en el mismo archivo o en otro
        with open(file_path, 'w', encoding='utf-8') as file:
            json.dump(sorted_data, file, indent=4)

        print(f"Ordenado y guardado: {filename}")

Ordenado y guardado: mandiant.json
Ordenado y guardado: paloaltounit42.json
Ordenado y guardado: crowdstrike.json
Ordenado y guardado: nsa.json
Ordenado y guardado: thales.json
Ordenado y guardado: microsoft.json
Ordenado y guardado: symantec.json
Ordenado y guardado: recorded_future.json
Ordenado y guardado: talosgroup.json
Ordenado y guardado: cert_ua.json
Ordenado y guardado: mitre.json
Ordenado y guardado: kaspersky.json
Ordenado y guardado: tencent.json
Ordenado y guardado: microsoft_old.json
Ordenado y guardado: secureworks.json
Ordenado y guardado: unk.json
Ordenado y guardado: fireeye.json
Ordenado y guardado: 360.json


In [47]:
# Order taxonomies alphabetically
import os
import json

# Rutas de las carpetas
taxonomies_dir = '1-taxonomies/'
unkFile = "unk.json"
dataUnk = load_or_create_json(taxonomies_dir+unkFile)
temp_data0 = {}


for d in dataUnk:
    if d == "Medusa (Note: Medusa should not be confused with a similarly named RaaS":
        temp_data0["Medusa"] = normalize_string("Medusa")
    else:
        if d == "Operation Blockbuster: Breach of Sony Pictures Entertainment":
            temp_data0["Operation Blockbuster"] = normalize_string("Operation Blockbuster")
        else:
            if d == "Subgroup: Andariel":
                temp_data0["Subgroup Andariel"] = normalize_string("Subgroup Andariel")
            else:
                if d == "Subgroup: Bluenoroff":
                    temp_data0["Subgroup Bluenoroff"] = normalize_string("Subgroup Bluenoroff")
                else:
                    temp_data0[d] = normalize_string(d)



with open(taxonomies_dir+unkFile, 'w', encoding='utf-8') as file0:
    json.dump(temp_data0, file0, ensure_ascii=False, indent=4)

# NORMALIZATION: Remove duplicates from unk.json

In [48]:

import json
import csv
import re
import os


def contains_name(data, name):
    # Convert everything to string and search recursively
    if isinstance(data, dict):
        return any(contains_name(v, name) or name in k for k, v in data.items())
    elif isinstance(data, list):
        return any(contains_name(v, name) for v in data)
    else:
        return name in str(data)

def removeUnkDuplicates():
    # Rutas de las carpetas
    taxonomies_dir = '1-taxonomies/'
    unkFile = "unk.json"
    
    dataUnk = load_or_create_json(taxonomies_dir+unkFile)
    #temp_data0 = data0.copy()
    
    temp_data0 = []
    
    
    # Lista todos los archivos en la carpeta
    for filename in os.listdir(taxonomies_dir):
        if filename.endswith('.json') and filename != unkFile:
            file_path = os.path.join(taxonomies_dir, filename)
    
            with open(file_path, 'r', encoding='utf-8') as file:
                dataTaxonomy = json.load(file)
    
                for d in dataTaxonomy:
                    value_to_remove = normalize_string(d)
                    
                    #print(value_to_remove)
                    
                    # Remove key-value pairs where the value matches the target
                    dataUnk = {k: v for k, v in dataUnk.items() if v != value_to_remove}
                    
    
    #print(dataUnk)
    with open(taxonomies_dir+unkFile, 'w', encoding='utf-8') as file0:
        json.dump(dataUnk, file0, ensure_ascii=False, indent=4)

removeUnkDuplicates()

# NORMALIZATION: Remove duplicates from taxonomies

In [49]:
import os
import json

# Working directory
folder = "1-taxonomies"

fireeye_file = os.path.join(folder, "fireeye.json")
exclude_file = os.path.join(folder, "unk.json")

# Function to load JSON
def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

# Function to save JSON
def save_json(path, data):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

# Load fireeye.json
fireeye = load_json(fireeye_file)

# Collect all keys and values from other JSON files
others_keys_values = set()

for filename in os.listdir(folder):
    if not filename.endswith(".json"):
        continue
    full_path = os.path.join(folder, filename)

    # Skip fireeye.json and unk.json
    if full_path in [fireeye_file, exclude_file]:
        continue

    # Load the other JSON file
    other_json = load_json(full_path)

    # Add keys and values to the comparison set
    others_keys_values.update(other_json.keys())
    others_keys_values.update(other_json.values())

# Filter fireeye.json by removing duplicates
filtered_fireeye = {
    k: v for k, v in fireeye.items()
    if k not in others_keys_values and v not in others_keys_values
}

# Save the cleaned fireeye.json
save_json(fireeye_file, filtered_fireeye)

print("✓ fireeye.json has been cleaned. Duplicate entries have been removed.")






# Files to modify
crowdstrike_path = os.path.join(folder, "crowdstrike.json")
kaspersky_path = os.path.join(folder, "kaspersky.json")
# ---- Remove from crowdstrike.json ----
crowdstrike = load_json(crowdstrike_path)

if "TEMP.PITTYTIGER" in crowdstrike:
    del crowdstrike["TEMP.PITTYTIGER"]

if "TEMP.PITTYTIGER" in crowdstrike.values():
    crowdstrike = {k: v for k, v in crowdstrike.items() if v != "TEMP.PITTYTIGER"}

save_json(crowdstrike_path, crowdstrike)

# ---- Remove from kaspersky.json ----
kaspersky = load_json(kaspersky_path)

if "SYKIPOT" in kaspersky:
    del kaspersky["SYKIPOT"]

if "SYKIPOT" in kaspersky.values():
    kaspersky = {k: v for k, v in kaspersky.items() if v != "SYKIPOT"}

save_json(kaspersky_path, kaspersky)

print("✓ Removed TEMP.PITTYTIGER from crowdstrike.json and SYKIPOT from kaspersky.json")

✓ fireeye.json has been cleaned. Duplicate entries have been removed.
✓ Removed TEMP.PITTYTIGER from crowdstrike.json and SYKIPOT from kaspersky.json


# ATTRIBUTION: Add Mappings

In [50]:
import requests, json, ast,os
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import pandas as pd


def updateFile(dst_path_file,proccessed_json):
        # Check if the file exists
    if os.path.exists(dst_path_file):
        # If it exists, you might want to load and update it, or just overwrite
        with open(dst_path_file, 'r', encoding='utf-8') as f:
            existing_data = json.load(f)
            existing_data.update(proccessed_json)
    else:
        existing_data = proccessed_json

    # Write the data to the file
    with open(dst_path_file, 'w', encoding='utf-8') as f:
        json.dump(existing_data, f, ensure_ascii=False, indent=4)
        

        
        
path_raw_data = "0-raw-data/"
path_proccesed_data = "2-1-mappings/"
sufix = "_mapping"

cleanFolderOrCreateIt(path_proccesed_data)


#####################################################################################
# ORKL
#####################################################################################

filename = "orkl_tas.json"
dst_filename = "orkl"+sufix+".json"
src_path_file = path_raw_data + filename
dst_path_file = path_proccesed_data + dst_filename

with open(src_path_file) as f:
    data = json.load(f)
    
sources = []
for ta in data:    
    sources.append(ta["source_name"].split(":")[0])

    
for s in sources:
    proccessed_json = {}
    dst_path_file = path_proccesed_data + s.lower() + sufix + ".json"
    
    for ta in data:
        source_name = ta["source_name"].split(":")[1]
        ta_source = ta["source_name"].split(":")[0]
        
        dest_source = ta_source
        
        if dest_source == "MISPGALAXY":
            dest_source = "UNK"
            #print(dest_source)
        aliases = []
        if "aliases" in ta:
            aliases = ta["aliases"]
            #print(type(aliases))
            
            
            if aliases == None: aliases = []
        
            temp_aliases = []
            for a in range(0,len(aliases)):
                if aliases[a] != "":
                    temp_aliases.append("UNK:"+aliases[a])
            aliases = temp_aliases
        
        
        ##########################################
        # SET ALL VENDOR NAMES TO UNK
        dest_source = "UNK"
        ##########################################
        
        if s == ta_source:
            #normalized_name = source_name.upper().replace(" ","").replace("-","")
            if aliases != []:
                while 'UNK:' in aliases:
                    aliases.remove('UNK:')
                proccessed_json[dest_source+":"+source_name] = {
                    "alias": aliases,
                    "source": "ORKL"
                }
        
    updateFile(dst_path_file,proccessed_json)
        
        
#####################################################################################
# MICROSOFT
#####################################################################################
    
filename = "MicrosoftMapping.json"
dst_filename = filename.split(".")[0]+sufix+"."+filename.split(".")[1]

src_path_file = path_raw_data + filename
dst_path_file = path_proccesed_data +filename

with open(src_path_file) as f:
    data = json.load(f)
    
proccessed_json = {}
for ta in data:
    source_name = "MICROSOFT:"+ta["Threat actor name"]
    #aliases = ast.literal_eval(ta["Other names"])
    aliases = ta["Other names"].split(",")
    #print(aliases)
    
    temp_aliases = []
    for a in range(0,len(aliases)):
        if aliases[a].strip() != "":
            temp_aliases.append("UNK:"+aliases[a].strip())
    aliases = temp_aliases
    

    if aliases != []:
        while 'UNK:' in aliases:
            aliases.remove('UNK:')
        proccessed_json[source_name] = {
            "alias": aliases,
            "source": "MSFT"
        }

updateFile(dst_path_file,proccessed_json)
        

#####################################################################################
# MISP
#####################################################################################

filename = "misp-threat-actor.json"
dst_filename = "mispgalaxy"+sufix+"."+filename.split(".")[1]
src_path_file = path_raw_data + filename
dst_path_file = path_proccesed_data +dst_filename

with open(src_path_file) as f:
    data = json.load(f)
    data = data["values"]
    
proccessed_json = {}
for ta in data:
    source_name = "UNK:"+ta["value"]
    aliases = []
    if "meta" in ta and "synonyms" in ta["meta"]:
        aliases = ta["meta"]["synonyms"]
    
    #print(aliases)
    #print(type(aliases))
    temp_aliases = []
    for a in range(0,len(aliases)):
        if aliases[a].strip() != "":
            temp_aliases.append("UNK:"+aliases[a].strip())
    aliases = temp_aliases
    
    
    if aliases != []:
        while 'UNK:' in aliases:
            aliases.remove('UNK:')
        
        proccessed_json[source_name] = {
            "alias": aliases,
            "source": "MISP"
        }

updateFile(dst_path_file,proccessed_json)
        
        
        
#####################################################################################
# MALPEDIA
#####################################################################################

filename = "malpedia_tas.json"
dst_filename = "malpedia" + sufix + "." + filename.split(".")[1]
src_path_file = path_raw_data + filename
dst_path_file = path_proccesed_data + dst_filename

with open(src_path_file) as f:
    data = json.load(f)
    
proccessed_json = {}
for ta in data:
    source_name = "UNK:"+ta
    aliases = data[ta]
    
    
    temp_aliases = []
    for a in range(0,len(aliases)):
        if aliases[a].strip() != "":
            temp_aliases.append("UNK:"+aliases[a].strip())
    aliases = temp_aliases
    
    
    if aliases != []:
        while 'UNK:' in aliases:
            aliases.remove('UNK:')
        proccessed_json[source_name] = {
            "alias": aliases,
            "source": "MALP"
        }

updateFile(dst_path_file,proccessed_json)
    


    
#####################################################################################
# APT Groups and Operations Excel (https://docs.google.com/spreadsheets/d/1H9_xaxQHpWaa4O_Son4Gx0YOIzlcBWMsdvePFX68EKU/edit?gid=361554658#gid=361554658)
#####################################################################################

filename = "APT Groups and Operations.xlsx"
dst_filename = "APT-Groups-and-Operations-excel"+sufix+".json"
src_path_file = path_raw_data + filename
dst_path_file = path_proccesed_data +dst_filename


# Load a specific sheet by name
#df_china = pd.read_excel('src_path_file', sheet_name='China')
proccessed_json = {}
for x in range(1,10):
    # Or load a sheet by index (0-based)
    df = pd.read_excel(src_path_file, sheet_name=x, header=1)
    #print(df)
    
    # Find the index of "Common Name"
    start_idx = df.columns.get_loc("Common Name")

    # Find the first column containing "operation" (case-insensitive)
    operation_idx = None
    for i, col in enumerate(df.columns):
        if "operation" in col.lower():
            operation_idx = i
            break

    # Determine the range of columns to process
    if operation_idx is None:
        columns_to_process = df.columns[start_idx + 1:]
    else:
        columns_to_process = df.columns[start_idx + 1:operation_idx]

    # Function to sanitize filenames
    def sanitize_filename(name):
        name = name.lower()
        # Special case: if name matches certain criteria, assign fixed filename
        if "cisco" in name:
            filename = "cisco.json"
        else:
            # Remove or replace problematic characters
            name = re.sub(r'[^\w\s-]', '', name)  # Remove special characters except underscores and hyphens
            name = re.sub(r'\s+', '_', name)      # Replace spaces with underscores
            filename = f"{name}.json"
        return filename

    # For each column, list the non-NaN elements
    non_nan_elements = {col: df[col].dropna().tolist() for col in df.columns}

    # Directory where to save JSON files
    output_dir = path_proccesed_data  # change this to your desired directory
    
   
        
    #print(columns_to_process)
    

    for index, row in df.iterrows():
        common_name = row['Common Name']
        # Skip if Common Name is NaN
        if pd.isna(common_name):
            continue

        # Convert row to a dictionary for easier processing
        row_dict = row.to_dict()
        
        # Assuming 'row' is a dict representing your row data
        common_name_value = str(row.get('Common Name', '')).strip()

        # Collect all other column values for comparison
        other_values = [
            str(row.get(c, '')).strip()
            for c in columns_to_process
            if c != 'Common Name' and not pd.isna(row.get(c))
        ]

        # Check if common_name_value matches any other column value
        if common_name_value not in other_values:
            source_name = f"UNK:{common_name_value}"
            aliases = []

            # Use other columns as aliases
            for c in columns_to_process:
                if c != 'Common Name' and not pd.isna(row.get(c)):
                    v_str = str(row[c]).strip()
                    # Handle splitting on comma
                    if ',' in v_str:
                        parts = [part.strip() for part in v_str.split(',')]
                        for part in parts:
                            # Check for '\n' in each part
                            if '\n' in part:
                                first_part = part.split('\n')[0].strip()
                            else:
                                first_part = part
                            # Añadir 'UNK:' al alias
                            aliases.append(f"UNK:{first_part}")
                    else:
                        # No comma, handle '\n'
                        if '\n' in v_str:
                            first_part = v_str.split('\n')[0].strip()
                        else:
                            first_part = v_str
                        # Añadir 'UNK:' al alias
                        aliases.append(f"UNK:{first_part}")
             
            if aliases != []:
                while 'UNK:' in aliases:
                    aliases.remove('UNK:')
                # Update the JSON
                proccessed_json[source_name] = {
                    "alias": aliases,
                    "source": "APTGO"
                }           
                #print(f"Updated {source_name}:{aliases}")
                updateFile(dst_path_file,proccessed_json)
            
        else:
    
            # Iterate through each column
            for col in columns_to_process:
                if col == 'Common Name':
                    continue  # skip self
                value = row[col]
                if pd.isna(value):
                    continue
                # Check if value matches the Common Name
                if str(value).strip() == str(common_name).strip():
                    # Prepare filename
                    filename = dst_filename
                    file_path = os.path.join(output_dir, filename)

                    col_name = col.upper()
                    # Prepare the JSON structure
                    if "OTHER NAME" in col.upper():
                        col_name = "UNK"
                    if "(" in col_name:
                        col_name = col_name.split("(")[0].strip()
                    
                    
                    ##########################################
                    # SET ALL VENDOR NAMES TO UNK
                    col_name = "UNK"
                    ##########################################
                    
                    
                    source_name = f"{col_name}:{common_name}"
                    #aliases = [str(v) for c, v in row.items() if c != 'Common Name' and not pd.isna(v) and str(v).strip() != str(common_name).strip()]
                    aliases = []

                    for c, v in row.items():
                        if c != 'Common Name' and c in columns_to_process and not pd.isna(v):
                            v_str = str(v).strip()
                            if v_str != str(common_name).strip():
                                # Split on comma first
                                if ',' in v_str:
                                    parts = [part.strip() for part in v_str.split(',')]
                                    for part in parts:
                                        # Check if '\n' is in the part
                                        if '\n' in part:
                                            first_part = part.split('\n')[0].strip()
                                        else:
                                            first_part = part
                                        # Añadir 'UNK:' al alias
                                        aliases.append(f"UNK:{first_part}")
                                else:
                                    # No comma, check for '\n'
                                    if '\n' in v_str:
                                        first_part = v_str.split('\n')[0].strip()
                                    else:
                                        first_part = v_str
                                    # Añadir 'UNK:' al alias
                                    aliases.append(f"UNK:{first_part}")
                    
                    
                    
                    if aliases != []:
                        while 'UNK:' in aliases:
                            aliases.remove('UNK:')
                        # Update the JSON
                        proccessed_json[source_name] = {
                            "alias": aliases,
                            "source": "APTGO"
                        }

                        #print(f"Updated {dst_path_file}")
                        updateFile(dst_path_file,proccessed_json)

                        
                        
#####################################################################################
# dict_tas_paloalto.json
#####################################################################################

filename = "dict_tas_paloalto.json"
dst_filename = "paloalto_mapping.json"
src_path_file = path_raw_data + filename
dst_path_file = path_proccesed_data + dst_filename

with open(src_path_file) as f:
    data = json.load(f)

proccessed_json = {}
for ta in data:
    source_name = "PALOALTOUNIT42:"+ta
    aliases = data[ta]["Also Known As"]
    
    
    temp_aliases = []
    for a in range(0,len(aliases)):
        if aliases[a].strip() != "":
            temp_aliases.append("UNK:"+aliases[a].strip())
    aliases = temp_aliases
    
    
    if aliases != []:
        while 'UNK:' in aliases:
            aliases.remove('UNK:')
        proccessed_json[source_name] = {
            "alias": aliases,
            "source": "PALOALTOUNIT42"
        }

updateFile(dst_path_file,proccessed_json)

##################################################################################





        
print("Done")

Done


# ATTRIBUTION: Attribute Mappings

In [51]:
import os
import json


# Función para buscar claves que contienen ese valor
def buscar_clave_por_valor(diccionario, valor):
    claves = []
    
    #print(diccionario.keys())
    # Iterate through outer dictionary
    for outer_key, inner_dict in diccionario.items():
        # Iterate through inner dictionary
        for inner_key, value in inner_dict.items():
            
            if normalize_string(value) == normalize_string(valor):

                return normalize_string(outer_key)
            
    return None



# Rutas de las carpetas
mappings_dir = '2-1-mappings'
taxonomies_dir = '1-taxonomies'
output_dir = '2-2-attributed_mappings'

cleanFolderOrCreateIt(output_dir)

# Cargar todos los archivos de taxonomías en un diccionario
taxonomies = {}
for filename in os.listdir(taxonomies_dir):
    
    if filename.endswith('.json'):
        filepath = os.path.join(taxonomies_dir, filename)
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
            # La clave del archivo será la fuente, por ejemplo, 'CROWDSTRIKE'
            source_name = os.path.splitext(filename)[0]
            #taxonomies[source_name] = normalize_string(data)
            taxonomies[source_name] = data
            

#print(taxonomies)
#print(type(taxonomies["tales"]))
# Crear un diccionario para toda la colección actualizada
total_mappings = {}


# Procesar cada archivo en mappings
for filename in os.listdir(mappings_dir):
    if filename.endswith('.json'):
        filepath = os.path.join(mappings_dir, filename)
        with open(filepath, 'r', encoding='utf-8') as f:
            mappings = json.load(f)

        # La fuente es el nombre del archivo sin extensión
        #source = os.path.splitext(filename)[0]+".json"
        #with open(taxonomies_dir+"/"+source, 'r') as f:
        #    taxonomies_map = json.load(f)
        
        
        # Procesar cada entrada en el mapeo
        for key, value in mappings.items():
            name = key
            #print(name)
            
            
            if name.startswith("UNK:") or name.startswith("MISPGALAXY:"):
                normalized_name = normalize_string(name.split(":")[1])
                replacement = buscar_clave_por_valor(taxonomies, normalized_name)
                #print(f"{replacement}")
                if replacement:
                    name = f"{replacement}:{normalized_name}"
                
                
                
            
            aliases = value.get('alias', [])
            source = value.get("source", "")
            updated_aliases = []
            
            for alias in aliases:
                alias_name = alias.split(":")[1]
                #normalized_alias = alias_name.strip().upper().replace(" ","").replace("-","").replace("/","").replace("\n","")
                normalized_alias = normalize_string(alias_name)
                
                if alias.startswith("UNK:") or alias.startswith("MISPGALAXY:"):

                    replacement = buscar_clave_por_valor(taxonomies, normalized_alias)
                    
                    

                    #print(f"{replacement}")
                    if replacement:
                        #new_alias = f"{replacement}:{alias_name}"
                        new_alias = f"{replacement}:{alias_name}:{normalize_string(source)}"
                        
                        updated_aliases.append(new_alias)
                    else:
                        # Si no se encuentra en la taxonomía, mantener igual o poner otra lógica
                        updated_aliases.append(f"{alias}:{source}")
                else:
                    updated_aliases.append(alias)
            # Guardar en el diccionario final
            if updated_aliases != [] and updated_aliases != ['UNK:']:
                '''total_mappings[name] = {
                    "alias": updated_aliases,
                    "source": source
                }'''
                # Remove 'UNK:' from the list
                while 'UNK:' in updated_aliases:
                    updated_aliases.remove('UNK:')
                
                if name in total_mappings:
                    total_mappings[name]["alias"] = total_mappings[name]["alias"] + updated_aliases
                    #total_mappings[name]["source"] = list(set(total_mappings[name]["source"] + [source]))
                else:
                    # Puedes crear la entrada si no existe
                    total_mappings[name] = {
                        "alias": updated_aliases,
                        #"source": [source]
                    }

# Guardar toda la información en un solo archivo en la carpeta de salida
output_path = os.path.join(output_dir, 'mappings.json')
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(total_mappings, f, ensure_ascii=False, indent=4)

print(f"Archivo consolidado guardado en {output_path}")



Archivo consolidado guardado en 2-2-attributed_mappings/mappings.json


In [52]:
import json, os
from collections import defaultdict
import networkx as nx
import matplotlib.pyplot as plt
import csv
import os, json
import ast

# Hacer fixes a los mappings
# Manual fix of typos found
# For each namecluster, we will add the reports and aliases associated found on ORKL

def replace_typo(s):
    
    if s == "KIMSUKI" or s == "KIMSUky":
        return "KIMSUKY"
    if s == "AGONIZNIGSERPENS":
        return "AGONIZINGSERPENS"
    if s == "BLACKENERGY(GROUP)" or s == "BLACKENERGYGROUP":
        return "BLACKENERGY"
    if s == "BUDMINERCYBERESPIONAGEGROUP":
        return "BUDMINER"
    if s == "C0D0SO" or s == "C0D0SO0" or s == "C0D0S0" or s == "CODOSOTEAM":
        return "CODOSO" 
    if s == "CALISTO" or s == "CALLISTO GROUP" or s == "CALLISTOGROUP":
        return "CALLISTO"
    if s == "DECEPTICONS":
        return "DECEPTIKONS"
    if s == "KEYBOY":
        return "KEYBOYS"
    if s == "DESERTFALCON":
        return "DESERTFALCONS"
    if s == "GAZAHACKERSTEAM":
        return "GAZAHACKERTEAM"
    if s == "LAPSUS\$":
        return "LAPSUS"
    if s == "NEMIN":
        return "NEMIM"
    if s == "PARASITOO" or s == "PARISITE" or s == "PARASTOO":
        return "PARASITE"
    if s == "QUDEDAGH" or s == "QUEDAGH GROUP":
        return "QUEDAGH"
    if s == "RANSOMED.VC" or s == "RANSOMED GROUP" or s == "RANSOMVC" or s == "RANSOM VC" or s == "RANSOMEDGROUP":
        return "RANSOMEDVC"
    if s == "REDBALDNIGHT":
        return "REDBALDKNIGHT"
    if s == "SALMONTYPHOO":
        return "SALMONTYPHOON"
    if s == "ABRAHAM'SAX":
        return "ABRAHAMSAX"    
    if s == "ADMIN@338":
        return "ADMIN338"    
    if s == "ALPHVM":
        return "ALPHV"    
    if s == "DYNCCALC":
        return "DYNCALC"    
    if s == "GUARDIANOFPEACE":
        return "GUARDIANSOFPEACE"    
    if s == "INC.RANSOM" or s == "INCRANSOM":
        return "INCRANSOMWARE"    
    if s == "LAPSUS$":
        return "LAPSUS"    
    if s == "MAGNALLIUM":
        return "MAGNALIUM"    
    if s == "NAHRELBARD":
        return "NAHRELBARED"    
    if s == "SHATHAK":
        return "SHATAK"    
    if s == "TEAMTONTO":
        return "TONTOTEAM"    
    if s == "CARBANAKGROUP":
        return "CARBANAK"    
    if s == "COBALTGANG" or s == "COBALTGROUP":
        return "COBALT"    
    if s == "COMMENTGROUP":
        return "COMMENTCREW"    
    if s == "CONFUCIUSAPT":
        return "CONFUCIUS"    
    if s == "CONIMESTEAM":
        return "CONIMES"        
    if s == "CONTITEAM1":
        return "CONTI"    
    if s == "ELDERWOODGANG" or s == "ELDERWOODGROUP":
        return "ELDERWOOD"    
    if s == "EQUATIONGROUP":
        return "EQUATION"    
    if s == "GAMAREDONGROUP":
        return "GAMAREDON"    
    if s == "HIPPOTEAM":
        return "HIPPO"       
    if s == "INCRANSOMWAREGROUP":
        return "INCRANSOMWARE"    
    if s == "KARAKURTLAIR" or s == "KARAKURTTEAM":
        return "KARAKURT"    
    if s == "LAZARUSGROUP" or s == "THELAZARUSGROUP":
        return "LAZARUS"    
    if s == "MACHETEAPT":
        return "MACHETE"    
    if s == "MENUPASSTEAM":
        return "MENUPASS"       
    if s == "MIRAGETEAM":
        return "MIRAGE"    
    if s == "NEWSCASTERTEAM":
        return "NEWSCASTER"    
    if s == "OCEANLOTUSGROUP":
        return "OCEANLOTUS"    
    if s == "QUEDAGHGROUP":
        return "QUEDAGH"    
    if s == "RANCORGROUP":
        return "RANCOR"       
    if s == "REAPERGROUP":
        return "REAPER"    
    if s == "ROYALAPT":
        return "ROYAL"    
    if s == "SECTORJ04GROUP":
        return "SECTORJ04"    
    if s == "SILENCEGROUP":
        return "SILENCE"    
    if s == "TAILGATERTEAM":
        return "TAILGATER"       
    if s == "TURLAGROUP" or s == "TURLATEAM":
        return "TURLA"    
    if s == "UPSTEAM":
        return "UPS"    
    if s == "WHOISHACKINGTEAM":
        return "WHOISTEAM"    
    if s == "WINNTIGROUP":
        return "WINNTI"    
    if s == "XHUNTCAMPAIGN" or s == "XHUNTCAMPAIGN(PALOALTO)":
        return "XHUNT"       
    if s == "THEGORGONGROUP":
        return "GORGONGROUP"    
    if s == "THELAMBERTS":
        return "LAMBERTS"    
    if s == "THEMASK":
        return "MASK"    
    if s == "THESHADOWBROKERS":
        return "SHADOWBROKERS"    
    if s == "APTC38(QIANXIN)":
        return "APTC38"       
    if s == "DEPUTYDOG(IDEFENSE)":
        return "DEPUTYDOG"    
    if s == "DOGFISH(IDEFENSE)":
        return "DOGFISH"    
    if s == "EARTHPRETA(TRENDMICRO)":
        return "EARTHPRETA"    
    if s == "HIVE0081(IBM)":
        return "HIVE0081"    
    if s == "SECTORD01(NHSC)":
        return "SECTORD01"       
    if s == "SHARPDRAGON(CHKPT)":
        return "SHARPDRAGON"    
    if s == "SYRIANELECTRONICARMY(SEA)":
        return "SYRIANELECTRONICARMY"    
    if s == "TA505(MERGEDWINDRIKSPIDER)":
        return "TA505"    
    if s == "TG2884(SCWXCTU)":
        return "TG2884"    
    if s == "WIZARDSPIDER(DEV0193)":
        return "WIZARDSPIDER"       
    if s == "APT36APT36":
        return "APT36"    
    if s == "ENERGETICBEARBERSERKBEAR":
        return "ENERGETICBEAR"    
    if s == "TG2754(TENTATIVE)":
        return "TG2754"     
    #if s == "OPERATIONCLANDESTINEFOX,PARTDEUX":
    #    return ""    
    #if s == "*OPERATIONLOLZARUS":
    #    return ""    
  
    #    if s == "":
    #        return ""

    return s


src_path = '2-2-attributed_mappings/'
src_file = "mappings.json"

# Cargar el JSON desde el directorio especificado
with open(src_path + src_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

temp_data = {}
#data = cluster_info
#to_del = []
for d in data:
    
    #print(d)
    aliases = []
    for alias_info in data[d]["alias"]:
        alias = alias_info.split(":")[1]
        alias_vendor = alias_info.split(":")[0]
        source = alias_info.split(":")[2]
        new_alias = replace_typo(normalize_string(alias))
        
        alias_info = alias_vendor + ":" + new_alias + ":" +source
        aliases.append(alias_info)
        
    #data[d]["alias"] = aliases
    main_name = d.split(":")[1]
    main_vendor = d.split(":")[0]
    new_main_name = replace_typo(normalize_string(main_name))
    temp_data[main_vendor + ":" + new_main_name] = data[d]
    temp_data[main_vendor + ":" + new_main_name]["alias"] = aliases

data = temp_data

# Guardar toda la información en un solo archivo en la carpeta de salida
print(data)
with open(src_path + src_file, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(f"Archivo consolidado guardado en {src_path + src_file}")          
#cluster_info = data

{'UNK:COMMENTCREW': {'alias': ['CROWDSTRIKE:COMMENTPANDA:APTGO', 'UNK:PLAUNIT61398:APTGO', 'SECUREWORKS:TG8223:APTGO', 'MANDIANT:APT1:APTGO', 'FIREEYE:BROWNFOX:APTGO', 'TALOSGROUP:GROUP3:APTGO', 'UNK:GIF89A:APTGO', 'UNK:SHADYRAT:APTGO', 'UNK:SHANGHAIGROUP:APTGO', 'UNK:BYZANTINECANDOR:APTGO', 'MITRE:G0006:APTGO', 'MANDIANT:APT1:ORKL', 'FIREEYE:BROWNFOX:ORKL', 'UNK:BYZANTINECANDOR:ORKL', 'UNK:BYZANTINEHADES:ORKL', 'UNK:COMMENTCREW:ORKL', 'CROWDSTRIKE:COMMENTPANDA:ORKL', 'UNK:GIF89A:ORKL', 'TALOSGROUP:GROUP3:ORKL', 'UNK:OPERATIONOCEANSALT:ORKL', 'UNK:OPERATIONSEASALT:ORKL', 'UNK:OPERATIONSIESTA:ORKL', 'UNK:SHANGHAIGROUP:ORKL', 'SECUREWORKS:TG8223:ORKL']}, 'MANDIANT:APT2': {'alias': ['CROWDSTRIKE:PUTTERPANDA:APTGO', 'UNK:PLAUNIT61486:APTGO', 'SECUREWORKS:TG6952:APTGO', 'TALOSGROUP:GROUP36:APTGO', 'UNK:SEARCHFIRE:APTGO', 'MITRE:G0024:APTGO', 'UNK:4HCREW:MALP', 'MITRE:G0024:MALP', 'UNK:MSUPDATER:MALP', 'UNK:PLAUNIT61486:MALP', 'CROWDSTRIKE:PUTTERPANDA:MALP', 'UNK:SULPHUR:MALP', 'UNK:SEARCHFI

# ATTRIBUTION: Add singletons

In [53]:
import json
import os

# -------------------------------
# Función que busca un nombre exacto en el JSON
# -------------------------------
def contains_name_exact(data, name):
    if isinstance(data, dict):
        for k, v in data.items():
            if normalize_string(k) == name:
                return True
            if contains_name_exact(v, name):
                return True
        return False
    elif isinstance(data, list):
        return any(contains_name_exact(v, name) for v in data)
    else:
        return normalize_string(str(data)) == name

# -------------------------------
# Paths
# -------------------------------
taxonomies_dir = '1-taxonomies'
mappings_dir = '2-2-attributed_mappings/'
attrMappings = "mappings.json"

# Cargar el JSON
with open(os.path.join(mappings_dir, attrMappings), 'r', encoding='utf-8') as f:
    dataMappings = json.load(f)

# -------------------------------
# Buscar singletons
# -------------------------------
singletons = []

for filename in os.listdir(taxonomies_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(taxonomies_dir, filename)

        # Usar solo el nombre del archivo en mayúsculas
        filename_label = filename.split(".json")[0].upper()

        # Leer taxonomía
        with open(file_path, 'r', encoding='utf-8') as f:
            dataTaxonomy = json.load(f)

            # Cada clave del diccionario de la taxonomía
            for key in dataTaxonomy.keys():
                tempName = normalize_string(key)

                if not contains_name_exact(dataMappings, tempName):
                    # Guardar como singleton
                    singletons.append(f"{filename_label}:{key}")

# -------------------------------
# Guardar resultado
# -------------------------------
output_path = os.path.join(mappings_dir, 'singletons.json')
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(singletons, f, ensure_ascii=False, indent=4)

print("DONE")


DONE


# ATTRIBUTION: Add new names to taxonomies found in mappings 

In [54]:
import os
import json

# -------------------------------
# Función de normalización
# -------------------------------
def normalize_string(s):
    return s.replace(" ", "").replace("-", "").upper().strip().replace("/", "").replace("\n", "").replace("_", "")

# -------------------------------
# Paths
# -------------------------------
taxonomies_dir = "1-taxonomies"
mappings_file = "2-2-attributed_mappings/mappings.json"
singletons_file = "2-2-attributed_mappings/singletons.json"
unk_file = os.path.join(taxonomies_dir, "unk.json")

# -------------------------------
# Cargar todos los elementos de las taxonomías
# -------------------------------
tax_elements = set()
for filename in os.listdir(taxonomies_dir):
    if filename.endswith(".json"):
        file_path = os.path.join(taxonomies_dir, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            if isinstance(data, dict):
                for key in data.keys():
                    tax_elements.add(normalize_string(key))
            elif isinstance(data, list):
                for item in data:
                    tax_elements.add(normalize_string(item))

# -------------------------------
# Cargar singletons
# -------------------------------
with open(singletons_file, "r", encoding="utf-8") as f:
    singletons = json.load(f)

# -------------------------------
# Cargar mappings
# -------------------------------
with open(mappings_file, "r", encoding="utf-8") as f:
    mappings = json.load(f)

mappings_elements = set()
for k, v in mappings.items():
    mappings_elements.add(k)
    for alias in v.get("alias", []):
        mappings_elements.add(alias)



print(len(singletons))
print(len(mappings_elements))
# -------------------------------
# Verificar qué elementos no están en taxonomías
# -------------------------------
unk_elements = {}

# De singletons
for s in singletons:
    n = normalize_string(s)
    if n.split(":")[1] not in tax_elements and n.split(":")[1] not in unk_elements:
        unk_elements[s.split(":")[1]] = n.split(":")[1]
#print(unk_elements)

# De mappings
for elem in mappings_elements:
    n = normalize_string(elem)
    if n.split(":")[1] not in tax_elements and elem.split(":")[1] not in unk_elements:
        
        unk_elements[elem.split(":")[1]] = n.split(":")[1]
        #print(elem)

print(unk_elements)
# -------------------------------
# Guardar unk.json
# -------------------------------

updateFile(unk_file,unk_elements)


print(f"{len(unk_elements)} elements added to {unk_file}")


removeUnkDuplicates()

3253
6048
{'ACIDBOX': 'ACIDBOX', 'DARKRACE': 'DARKRACE', 'КИБЕРПАРТИЗАНЫ': 'КИБЕРПАРТИЗАНЫ', 'GROUPWITRE': 'GROUPWITRE', '?': '?', 'BL00DY': 'BL00DY', 'MANLINGFLOWER(MANLINGHUA)': 'MANLINGFLOWER(MANLINGHUA)', 'MEDUSA(NOTE': 'MEDUSA(NOTE', 'OFFICEMONKEYSTEMP.MONKEYS': 'OFFICEMONKEYSTEMP.MONKEYS', 'SNATCH': 'SNATCH', 'SIG25(NSA)': 'SIG25(NSA)', 'RYUK(ASFIN12)': 'RYUK(ASFIN12)', 'КІБЕРПАРТЫЗАНЫ': 'КІБЕРПАРТЫЗАНЫ', 'SUNFLOWER': 'SUNFLOWER', 'گنجشکدرنده': 'گنجشکدرنده', 'TA545': 'TA545', 'THEMX0NDAY': 'THEMX0NDAY', 'TA413(PROOFPOINT)': 'TA413(PROOFPOINT)', 'PHOSPHORUSMINTSANDSTORM': 'PHOSPHORUSMINTSANDSTORM', 'REVIL': 'REVIL', 'HÀNHTINHCOMPANYLIMITED': 'HÀNHTINHCOMPANYLIMITED', 'SUBGROUP': 'SUBGROUP', 'APTC01': 'APTC01', 'UNSAFE': 'UNSAFE', 'APTC44': 'APTC44', 'DUBNIUM(MICROSOFT)': 'DUBNIUM(MICROSOFT)', 'تبرابراهیم': 'تبرابراهیم', 'BOYUSEC–THEGUANGZHOUBOYUINFORMATIONTECHNOLOGYCOMPANY': 'BOYUSEC–THEGUANGZHOUBOYUINFORMATIONTECHNOLOGYCOMPANY', 'MYSTERIOUSMETADOR': 'MYSTERIOUSMETADOR', 'SHAMOON'

# CLUSTERING: Make clusters

In [55]:
import json, os
from collections import defaultdict
import networkx as nx
import matplotlib.pyplot as plt
import csv
import os, json
import ast, copy


# Union-Find (Disjoint Set) Data Structure
class UnionFind:
    def __init__(self):
        self.parent = {}
    
    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])  # Path compression
        return self.parent[x]
    
    def union(self, x, y):
        rootX = self.find(x)
        rootY = self.find(y)
        if rootX != rootY:
            self.parent[rootY] = rootX  # Union operation

    def add(self, x):
        if x not in self.parent:
            self.parent[x] = x


            
data = {}
attr_mappings_dir = '2-2-attributed_mappings'
output_dir = '3-nameclusters'

# Cargar el JSON desde el directorio especificado
with open(attr_mappings_dir + '/mappings.json', 'r', encoding='utf-8') as f:
    mappings = json.load(f)

# Your original JSON structure
#data = mappings.copy()

data = copy.deepcopy(mappings)
print(data)


clusters = {}

G = nx.Graph()
items = []

clusters_data = {}

for name in data:
    #print(name)
    mapping_name = name.split(":")[1]
    aliases = data[name]["alias"]
    

    if isinstance(aliases, str):
        #print("Es una cadena")
        aliases = [aliases]
    
    print(mapping_name)
    print(aliases)
    for alias in aliases:
        #items.append([name,a,source])
        
        print(mapping_name)
        print(alias)
        a = ":".join(alias.split(":")[:2])
        source = alias.split(":")[2]
        
        if name != None and a != None:
            G.add_edge(name, a, src = source)
            
            #G.add_edge(name, alias, label=str([linkInfo]), num_links=1)



# Add singletons                  
# Cargar el JSON desde el directorio especificado
with open(attr_mappings_dir + '/singletons.json', 'r', encoding='utf-8') as f:
    mappings = json.load(f)
    
data = copy.deepcopy(mappings)
print(data)
for name in data:
    #print(name)
    mapping_name = name.split(":")[1]
    vendor = [name.split(":")[0]]

    if not G.has_node(name):
        G.add_node(name)





# Obtener componentes conexas
components = list(nx.connected_components(G))

cluster_info = {}

for c_idx, component in enumerate(components):
    nc = "namecluster_"+str(c_idx)  # índice único para cada componente
    cluster_info[nc] = []  # inicializar lista para este cluster

    # Check if the component has size one
    if len(component) == 1:
        print("Added "+str(component))
        cluster_info[nc].append({"name": list(component)[0]})
        
    else:
        
        for nodeX in component:
            for nodeY in component:
                if nodeX != nodeY and G.has_edge(nodeX, nodeY):
                    source = G[nodeX][nodeY]["src"]
                    cluster_info[nc].append({"name": nodeX, "alias": nodeY, "source": source})

# Resultado
print(cluster_info)
            

#normalize_string(s)

source_path = "3-nameclusters/"
input_file = "nameclusters.json"

            
with open(source_path+input_file, 'w') as json_file:
    json.dump(cluster_info, json_file, indent=4)

print(f"Clusters saved to {source_path+input_file}")


{'UNK:COMMENTCREW': {'alias': ['CROWDSTRIKE:COMMENTPANDA:APTGO', 'UNK:PLAUNIT61398:APTGO', 'SECUREWORKS:TG8223:APTGO', 'MANDIANT:APT1:APTGO', 'FIREEYE:BROWNFOX:APTGO', 'TALOSGROUP:GROUP3:APTGO', 'UNK:GIF89A:APTGO', 'UNK:SHADYRAT:APTGO', 'UNK:SHANGHAIGROUP:APTGO', 'UNK:BYZANTINECANDOR:APTGO', 'MITRE:G0006:APTGO', 'MANDIANT:APT1:ORKL', 'FIREEYE:BROWNFOX:ORKL', 'UNK:BYZANTINECANDOR:ORKL', 'UNK:BYZANTINEHADES:ORKL', 'UNK:COMMENTCREW:ORKL', 'CROWDSTRIKE:COMMENTPANDA:ORKL', 'UNK:GIF89A:ORKL', 'TALOSGROUP:GROUP3:ORKL', 'UNK:OPERATIONOCEANSALT:ORKL', 'UNK:OPERATIONSEASALT:ORKL', 'UNK:OPERATIONSIESTA:ORKL', 'UNK:SHANGHAIGROUP:ORKL', 'SECUREWORKS:TG8223:ORKL']}, 'MANDIANT:APT2': {'alias': ['CROWDSTRIKE:PUTTERPANDA:APTGO', 'UNK:PLAUNIT61486:APTGO', 'SECUREWORKS:TG6952:APTGO', 'TALOSGROUP:GROUP36:APTGO', 'UNK:SEARCHFIRE:APTGO', 'MITRE:G0024:APTGO', 'UNK:4HCREW:MALP', 'MITRE:G0024:MALP', 'UNK:MSUPDATER:MALP', 'UNK:PLAUNIT61486:MALP', 'CROWDSTRIKE:PUTTERPANDA:MALP', 'UNK:SULPHUR:MALP', 'UNK:SEARCHFI

# CLUSTERING: Enrich with orkl reports and dates

In [56]:
# For each namecluster, we will add the reports and aliases associated found on ORKL

import matplotlib.pyplot as plt
import csv
import os, json
#import networkx as nx
import ast

#normalize_string(s)

source_path = "3-nameclusters/"
nc_file = "nameclusters.json"
reports_file = "0-raw-data/dates_list2.csv"
data_csvs_path = "Graphs-CSVs/"
output_file = "7-Enriched_nameclusters.json"


# Node2: 70db80bd-31b7-4581-accb-914cd8252913, Attributes2: {'num_aliases': 4, 
# 'aliases': 'Longhorn, the Lamberts, APT-C-39, PLATINUM TERMINAL, ', 'name': 'MISPGALAXY:Longhorn', 
# 'first_year': 2017}

if not os.path.exists(data_csvs_path): 
    os.makedirs(data_csvs_path) 

ncs = []
with open(source_path+nc_file,"r", encoding="utf-8") as f:
    try:
        ncs = json.load(f)
    except json.JSONDecodeError as e:
            print(f"Error en {file}: {e}")


            
data = []
#with open(source_path+reports_file, 'r') as file:
with open(reports_file, 'r') as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
        data.append(lines)
#print(data[1])
#['70db80bd-31b7-4581-accb-914cd8252913', '2017-04-10', 'Longhorn', '756f1b82-3280-4230-8faf-e59214d29549', 'Symantec']

dates_dict = {}

for d in range(1,len(data)):
    #print(data[d])
    if data[d][1] != "":
        report_date = int(data[d][1][:4])
        name = normalize_string(data[d][2])
        report_id = data[d][3]
        report_vendor = normalize_string(data[d][4])

        if name not in dates_dict:
            dates_dict[name] = [report_date,report_date,[report_vendor],[report_id]] # minDate, maxDate, vendor
        else:
            if dates_dict[name][0] > report_date:
                dates_dict[name][0] = report_date
            if dates_dict[name][1] < report_date:
                dates_dict[name][1] = report_date
            if report_vendor not in dates_dict[name][2]:
                dates_dict[name][2].append(report_vendor)
            if report_id not in dates_dict[name][3]:
                dates_dict[name][3].append(report_id)

#print(dates_dict)



for cluster in ncs:
    #print(ncs["namecluster_0"])

    #print(cluster)
    #print(len(ncs[cluster]))
    
    if len(ncs[cluster]) == 1 and not hasattr(ncs[cluster][0], 'alias'):
        name = normalize_string(association["name"].split(":")[1])
        vendor_name = normalize_string(association["name"].split(":")[0])
        
        if name in dates_dict:
            association["minDateName"] = dates_dict[name][0]
            association["maxDateName"] = dates_dict[name][1]
            association["reports_vendors_name"] = dates_dict[name][2]
            association["reports_name"] = (dates_dict[name][3])
        else:
            association["minDateName"] = 9999
            association["maxDateName"] = -1
            association["reports_vendors_name"] = []
            association["reports_name"] = []
    else:
        for association in ncs[cluster]:
            
            name = normalize_string(association["name"].split(":")[1])
            vendor_name = normalize_string(association["name"].split(":")[0])
            alias = normalize_string(association["alias"].split(":")[1])
            vendor_alias = normalize_string(association["alias"].split(":")[0])
            #source = normalize_string(association["source"])
    
        
            if name in dates_dict and alias in dates_dict:
                association["minDateName"] = dates_dict[name][0]
                association["maxDateName"] = dates_dict[name][1]
                association["minDateAlias"] = dates_dict[alias][0]
                association["maxDateAlias"] = dates_dict[alias][1]
    
                association["reports_vendors_name"] = dates_dict[name][2]
                association["reports_name"] = (dates_dict[name][3])
                
                association["reports_vendors_alias"] = dates_dict[alias][2]
                association["reports_alias"] = dates_dict[alias][3]
                    
            else:
                
                if name in dates_dict:
                    association["minDateName"] = dates_dict[name][0]
                    association["maxDateName"] = dates_dict[name][1]
                    association["reports_vendors_name"] = dates_dict[name][2]
                    association["reports_name"] = (dates_dict[name][3])
                else:
                    association["minDateName"] = 9999
                    association["maxDateName"] = -1
                    association["reports_vendors_name"] = []
                    association["reports_name"] = []
    
                if alias in dates_dict:
                    association["minDateAlias"] = dates_dict[alias][0]
                    association["maxDateAlias"] = dates_dict[alias][1]
                    association["reports_vendors_alias"] = dates_dict[alias][2]
                    association["reports_alias"] = dates_dict[alias][3]
                else:
                    association["minDateAlias"] = 9999
                    association["maxDateAlias"] = -1
                    association["reports_vendors_alias"] = []
                    association["reports_alias"] = []

with open(source_path+output_file, 'w') as json_file:
    json.dump(ncs, json_file, indent=4)
    print(f"{source_path+output_file} created.")

    


3-nameclusters/7-Enriched_nameclusters.json created.


# CLUSTERING: Make graph

In [57]:
import json,os
import networkx as nx
import matplotlib.pyplot as plt


# Step 1: Read the nameclusters.json file
output_dir = '3-nameclusters' # Define the output directory path

#input_file = "nameclusters.json"
input_file = "7-Enriched_nameclusters.json"

input_path = os.path.join(output_dir, input_file)

with open(input_path, 'r') as json_file:
    cluster_data = json.load(json_file)

# Step 2: Initialize a NetworkX graph
G = nx.Graph()
dict_tas = {}
dict_aliases = {}

# Step 3: Add nodes and edges to the graph
for cluster, items in cluster_data.items():

    if len(items) == 1:
        item = items[0]
        name = normalize_string(item['name']).split(":")[1]
        vendor = normalize_string(item['name']).split(":")[0]
        
        reports_name = []
        minDateName = 9999
        maxDateName = -1
        vendors = []
        reports_vendors_name = []
        reports_name = []
        
        
        G.add_node(name, label=name, vendors=vendor, num_vendors=len(vendor), firstYear=minDateName,
                           lastYear=maxDateName, reports_vendors=reports_vendors_name, 
                           reports_list=reports_name,num_reports=len(reports_name),clusterID=cluster)
    else:
        for item in items:
            name = normalize_string(item['name']).split(":")[1]
            vendor1 = normalize_string(item['name']).split(":")[0]
            minDateName = item['minDateName']
            maxDateName = item['maxDateName']
            
            alias = normalize_string(item['alias']).split(":")[1]
            vendor2 = normalize_string(item['alias']).split(":")[0]
            minDateAlias = item['minDateAlias']
            maxDateAlias = item['maxDateAlias']
            
            source = item['source']
            
            reports_vendors_name = item['reports_vendors_name']
            reports_name = item['reports_name']
            
            reports_vendors_alias = item['reports_vendors_alias']
            reports_alias = item['reports_alias']
            
            
            
            # Add nodes with the 'label' as name, and 'alias' and 'source' as attributes
            #if name in dict_tas:
            if name in G.nodes:
                minDate = minDateName
                maxDate = maxDateName
                vendor = vendor1
                
    
                if vendor != "UNK" and vendor not in G.nodes[name]['vendors']:
                    G.nodes[name]['vendors'].append(vendor)
                G.nodes[name]['num_vendors']=len(G.nodes[name]['vendors'])
       
                
                if G.nodes[name]['firstYear'] > minDate:
                    G.nodes[name]['firstYear'] = minDate
                if G.nodes[name]['lastYear'] < maxDate:
                    G.nodes[name]['lastYear'] = maxDate
                    
                G.nodes[name]['reports_vendors'] = list(set(G.nodes[name]['reports_vendors']+reports_vendors_name))
                G.nodes[name]['reports_list'] = list(set(G.nodes[name]['reports_list']+reports_name))
                
                G.nodes[name]['num_reports']=len(G.nodes[name]['reports_list'])
                
            else:
                vendors = []
                if vendor1 != "UNK":
                    vendors.append(vendor1)    
                    
                G.add_node(name, label=name, vendors=vendors, num_vendors=len(vendors), firstYear=minDateName,
                           lastYear=maxDateName, reports_vendors=reports_vendors_name, 
                           reports_list=reports_name,num_reports=len(reports_name),clusterID=cluster)
                #print(f"Created node: {name}, {G.nodes[name]}")
    
                
                
            # Add nodes with the 'label' as name, and 'alias' and 'source' as attributes
            if item['alias'] != "UNK:":
                #if alias in dict_tas:
                if alias in G.nodes:
                    minDate = minDateAlias
                    maxDate = maxDateAlias
                    vendor = vendor2
                    
                    
                    
                    
                    if vendor != "UNK" and vendor not in G.nodes[alias]['vendors']:
                        G.nodes[alias]['vendors'].append(vendor)
                    G.nodes[alias]['num_vendors']=len(G.nodes[alias]['vendors'])
    
                    
                    
                    if G.nodes[alias]['firstYear'] > minDate:
                        G.nodes[alias]['firstYear'] = minDate
                    if G.nodes[alias]['lastYear'] < maxDate:
                        G.nodes[alias]['lastYear'] = maxDate
    
                    G.nodes[alias]['reports_vendors'] = list(set(G.nodes[alias]['reports_vendors']+reports_vendors_alias))
                    G.nodes[alias]['reports_list'] = list(set(G.nodes[alias]['reports_list']+reports_alias))
    
                    G.nodes[alias]['num_reports']=len(G.nodes[alias]['reports_list'])
    
                else:
                    vendors = []
                    if vendor2 != "UNK":
                        vendors.append(vendor2)    
                    
                    
                    
                    G.add_node(alias, label=alias, vendors=vendors, num_vendors=len(vendors), firstYear=minDateAlias,
                               lastYear=maxDateAlias, reports_vendors=reports_vendors_alias, 
                               reports_list=reports_alias,num_reports=len(reports_alias),clusterID=cluster)
                    #print(f"Created node: {name}, {G.nodes[name]}")
    
                    
            if item['alias'] != "UNK:":
                #linkInfo =[name, alias, source]
                linkInfo = source
                if G.has_edge(name,alias):
                    #G[name][alias]["num_aliases_comunes"] += 1
                    
                    #linksList = str(list(set(ast.literal_eval(G[name][alias]["label"]) + linkInfo)))
                    #linksList = list(set(G[name][alias]["label"] + linkInfo))
                    #linksList.append(linkInfo)
                    linksList = str(list(set(ast.literal_eval(G[name][alias]["label"]) + [linkInfo] )))
                    
                    
                    G[name][alias]["label"] = linksList
                    
                    G[name][alias]["num_links"] += 1 
                    #print(f"Edited edge: {G[name][alias]}")
    
                else:
                    #G.add_edge(name, alias, num_aliases_comunes=1, source=[linkInfo], num_links=1)
                    #G.add_edge(name, alias, num_aliases_comunes=1, label=str(linkInfo), num_links=1)
                    G.add_edge(name, alias, label=str([linkInfo]), num_links=1)
                    #print(f"Created edge: {G[name][alias]}")
                
            if item['alias'] != "UNK:" and name != "":
                if name not in dict_tas:
                    dict_tas[name] = []
                dict_tas[name].append(alias)
            
            if item['alias'] != "UNK:" and name != "":
                if alias not in dict_aliases:
                    dict_aliases[alias] = []
                dict_aliases[alias].append(name)
        
        
        # Create edges between related nodes
        #G.add_edge(name, alias)
        #G.add_edge(name, source)
        #G.add_edge(alias, source)

graph = G


nodes_list = list(G.nodes())
#print(graph.nodes[2105])
for node in G.nodes:
    
    if G.nodes[node]["vendors"] == []:
        G.nodes[node]["vendors"] = ["UNK"]
        #print(G.nodes[node]["vendors"])
    if G.nodes[node]["vendors"] == ["MICROSOFTTAXONOMY"]:
        G.nodes[node]["vendors"] = ["MICROSOFT"]
        

# Step 4: Save the graph to a GML file
gml_output_path = os.path.join(output_dir, "graph.gml")
nx.write_gml(graph, gml_output_path)

print(f"GML file saved to {gml_output_path}")


GML file saved to 3-nameclusters/graph.gml
